In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('data/TSLA.csv')

# Convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set date as index
df.set_index('Date', inplace=True)

# Filter out data before 2016
df = df[df.index.year >= 2016]

# Calculate the end of week dates
df['Week'] = df.index.to_series().dt.isocalendar().week
df['Year'] = df.index.to_series().dt.isocalendar().year
df['Month'] = df.index.month  # Add Month column
df['DateStr'] = df.index.strftime('%y%m%d')
df['TradingWeek'] = (
    df['Year'].astype(str) + '-' 
    + df['Week'].astype(str).apply(lambda x: str(x).zfill(2))
)

# Filter out the end trading day of each week
weekly_df = df.groupby('TradingWeek').last()

# Calculate the last Friday's close price
weekly_df['PrevClose'] = weekly_df['Adj Close'].shift(1)

# Initialize the profit and investment column
weekly_df['Profit_No_Reinvest'] = 0.0
weekly_df['Investment_No_Reinvest'] = 1.0

weekly_df['Profit_Reinvest'] = 0.0
weekly_df['Investment_Reinvest'] = 1.0

strike_factor = 1.15
premium = 0.005
leverage = 3
for i in range(1, len(weekly_df)):
    prev_close = weekly_df.loc[weekly_df.index[i-1], 'Adj Close']
    this_close = weekly_df.loc[weekly_df.index[i], 'Adj Close']
    if this_close <= prev_close * strike_factor:
        profit = prev_close * premium
    else:
        profit = prev_close * premium - leverage * (this_close - prev_close * strike_factor)
    
    # Each week's profit is the investment times the premium
    weekly_df.loc[weekly_df.index[i], 'Profit_No_Reinvest'] = weekly_df.loc[weekly_df.index[i-1], 'Investment_No_Reinvest'] * profit

    # For reinvest, each week's profit is the investment times the premium, and then add the profit to the investment
    weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest'] = weekly_df.loc[weekly_df.index[i-1], 'Investment_Reinvest'] * profit
    weekly_df.loc[weekly_df.index[i], 'Investment_Reinvest'] = weekly_df.loc[weekly_df.index[i-1], 'Investment_Reinvest'] + weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest']

# Calculate the current value of 1 dollar investment in TSLA stock
weekly_df['Stock_Value'] = weekly_df['Close'] / weekly_df.loc[weekly_df.index[0], 'Close']

# Select only necessary columns
output_df = weekly_df[['DateStr', 'PrevClose', 'Close', 'Profit_No_Reinvest', 'Investment_No_Reinvest', 'Profit_Reinvest', 'Investment_Reinvest', 'Stock_Value']].reset_index()
output_df

In [59]:
# Initialize the profit and investment column
pd.set_option('display.max_rows', 500)

init_invest = 1.0
weekly_df['Profit_No_Reinvest'] = 0.0
weekly_df['Investment_No_Reinvest'] = init_invest # The initial investment is $1

weekly_df['Profit_Reinvest'] = 0.0
weekly_df['Investment_Reinvest'] = init_invest  # The initial investment is $1

strike_factor = 1.15  # Adjust strike factor to 1.3
premium = 0.0065
leverage = 5
for i in range(1, len(weekly_df)):
    prev_close = weekly_df.loc[weekly_df.index[i-1], 'Adj Close']
    this_close = weekly_df.loc[weekly_df.index[i], 'Adj Close']
    strike_price = prev_close * strike_factor
    # Calculate One Share Profit
    if this_close < strike_price:
        profit = prev_close * premium
    else:
         # calculate loss in the future
        profit = 0 #prev_close * premium - leverage * (this_close - strike_price) 
   
    # percentage profit, as we use $1 to evaluate each week. 
    weekly_option_return = profit / weekly_df.loc[weekly_df.index[i-1], 'Adj Close']
    weekly_stock_return = (this_close - prev_close) / prev_close
    # No Re-invest
    # Each week's profit is the calculated profit x initial_invest
    # Investment_No_Reinvest is the cum sum of each week profit
    weekly_df.loc[weekly_df.index[i], 'Profit_No_Reinvest'] = weekly_option_return + weekly_stock_return
    weekly_df.loc[weekly_df.index[i], 'Investment_No_Reinvest'] = (
        weekly_df.loc[weekly_df.index[i-1], 'Investment_No_Reinvest'] + # Previous Investment 
        weekly_df.loc[weekly_df.index[i], 'Profit_No_Reinvest'] # Option Trading Profit 
    )

    # Re-invest
    # For reinvest, each week's profit is the total value of all stocks at the end of the week times the premium
    weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest'] = (
        weekly_option_return * weekly_df.loc[weekly_df.index[i-1], 'Investment_Reinvest'] * weekly_stock_return
    )
    weekly_df.loc[weekly_df.index[i], 'Investment_Reinvest'] = (
        weekly_df.loc[weekly_df.index[i-1], 'Investment_Reinvest'] +
        weekly_df.loc[weekly_df.index[i], 'Profit_Reinvest']
    )
    

# Calculate the current value of 1 dollar investment in TSLA stock
weekly_df['Stock_Value'] = weekly_df['Close'] / weekly_df.loc[weekly_df.index[0], 'Close']

# Select only necessary columns
output_df = weekly_df[[
    'Profit_No_Reinvest',
    'Profit_Reinvest',
    'Investment_No_Reinvest', 
    'Investment_Reinvest', 
    'Stock_Value'
    ]].reset_index()
output_df


,TradingWeek,Profit_No_Reinvest,Profit_Reinvest,Investment_No_Reinvest,Investment_Reinvest,Stock_Value
0,2016-01,0.000000,0.000000e+00,1.000000,1.000000,1.000000
1,2016-02,-0.021983,-1.851423e-04,0.978017,0.999815,0.971517
2,2016-03,-0.005403,-7.735546e-05,0.972614,0.999738,0.959953
3,2016-04,-0.049535,-3.641351e-04,0.923078,0.999373,0.906161
4,2016-05,-0.143082,-9.716712e-04,0.779996,0.998402,0.770616
5,2016-06,-0.064595,-4.613772e-04,0.715402,0.997940,0.715829
6,2016-07,0.109387,6.673858e-04,0.824788,0.998608,0.789479
7,2016-08,0.149134,9.258313e-04,0.973922,0.999534,0.902085
8,2016-09,0.062715,3.652287e-04,1.036638,0.999899,0.952796
9,2016-10,0.038633,2.088424e-04,1.075271,1.000108,0.983412


## Optimize Trading Strategy

In [ ]:
# Earnings dates
earnings_dates = pd.to_datetime([
    '7/19/23', '4/19/23', '1/25/23', '10/19/22', '7/20/22', '4/20/22', '1/26/22', '10/20/21', 
    '7/26/21', '4/26/21', '1/27/21', '10/21/20', '7/22/20', '4/29/20', '1/29/20', '10/23/19', 
    '6/24/19', '4/24/19', '1/30/19', '10/24/18', '8/1/18', '5/2/18', '2/7/18', '11/1/17', 
    '8/2/17', '5/3/17', '2/22/17', '10/26/16', '8/3/16', '5/4/16'
])

# Function to get the first week in the data that is after each earnings date
def first_week_after_date(date):
    if date > weekly_data.index[-1]:
        return None
    return weekly_data[date:].index[0]

# Apply the function to the earnings dates
earnings_weeks = earnings_dates.map(first_week_after_date)

# Weeks that are both high return weeks and have earnings dates
high_return_earnings_weeks = weekly_data.loc[high_return_weeks.index.intersection(earnings_weeks)]

# Plot the weekly returns, highlighting the earnings weeks
plt.figure(figsize=(12, 6))
plt.plot(weekly_data.index, weekly_data['Weekly Return %'], label='All weeks')
plt.scatter(high_return_earnings_weeks.index, high_return_earnings_weeks['Weekly Return %'], 
            color='red', label='High return earnings weeks', marker='x', s=100)
plt.title('Weekly Returns of Tesla Shares')
plt.xlabel('Date')
plt.ylabel('Weekly Return %')
plt.legend()
plt.grid()
plt.show()